In [0]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [0]:
dc_listings = pd.read_csv('listings3.csv')
#dc_listings.columns

In [0]:
dc_listings = dc_listings[['accommodates', 'bedrooms', 'bathrooms','number_of_reviews','price']]

In [0]:
dc_listings['price']

0       $160.00
1       $350.00
2        $50.00
3        $95.00
4        $50.00
5        $99.00
6       $100.00
7       $100.00
8        $38.00
9        $71.00
10       $97.00
11       $55.00
12       $50.00
13       $99.00
14       $60.00
15       $52.00
16       $23.00
17      $200.00
18       $40.00
19          NaN
20          NaN
21          NaN
22          NaN
23          NaN
24          NaN
25          NaN
26          NaN
27          NaN
28          NaN
29          NaN
         ...   
3693        NaN
3694        NaN
3695        NaN
3696        NaN
3697        NaN
3698        NaN
3699        NaN
3700        NaN
3701        NaN
3702        NaN
3703        NaN
3704        NaN
3705        NaN
3706        NaN
3707        NaN
3708        NaN
3709        NaN
3710        NaN
3711        NaN
3712        NaN
3713        NaN
3714        NaN
3715        NaN
3716        NaN
3717        NaN
3718        NaN
3719        NaN
3720        NaN
3721        NaN
3722        NaN
Name: price, Length: 372

In [0]:
dc_listings['price'] = dc_listings['price'].apply(lambda x:x.replace('$',''))
dc_listings['price'] = dc_listings['price'].apply(lambda x:x.replace(',',''))
dc_listings['price'] = dc_listings['price'].astype('float')

In [0]:
dc_listings['price'].head()

0    160.0
1    350.0
2     50.0
3     95.0
4     50.0
Name: price, dtype: float64

In [0]:
dc_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 18
Data columns (total 5 columns):
accommodates         17 non-null float64
bedrooms             17 non-null float64
bathrooms            17 non-null float64
number_of_reviews    17 non-null float64
price                17 non-null float64
dtypes: float64(5)
memory usage: 816.0 bytes


In [0]:

dc_listings = dc_listings.dropna(axis=0)
dc_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 18
Data columns (total 5 columns):
accommodates         17 non-null float64
bedrooms             17 non-null float64
bathrooms            17 non-null float64
number_of_reviews    17 non-null float64
price                17 non-null float64
dtypes: float64(5)
memory usage: 816.0 bytes


In [0]:
train_rows  =  int(dc_listings.shape[0] * 0.66)
train = dc_listings.iloc[0:train_rows]
test = dc_listings.iloc[train_rows:]

In [0]:
features = list(train.columns)
features.remove('price')
features

['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews']

In [0]:
def predict_price(row,temp_df):
    distances = temp_df[features] - row
    squared = np.square(distances)
    dist = squared.sum(axis=1)
    temp_df['distance'] = dist**0.5
    new_df = temp_df.sort_values('distance')
    nearest_neighbor_prices = new_df['price'].iloc[0:5]
    predicted_price = nearest_neighbor_prices.mean()
    return predicted_price
   

In [0]:
for i,row in test[features].iterrows():
    predicted_price = predict_price(row,train)
    test.loc[i,'predicted_price'] = predicted_price

In [0]:
train.sort_values('distance')
test

,accommodates,bedrooms,bathrooms,number_of_reviews,price,predicted_price
12,2.0,1.0,1.0,0.0,50.0,67.6
13,2.0,0.0,1.0,0.0,99.0,67.6
15,2.0,1.0,1.0,1.0,52.0,67.6
16,1.0,1.0,1.0,1.0,23.0,67.6
17,2.0,1.0,1.0,0.0,200.0,67.6
18,4.0,2.0,1.0,0.0,40.0,100.8


In [0]:
mae = np.absolute(test['price'] - test['predicted_price']).sum()/test.shape[0]
print(test.shape[0])
print(mae)

6
50.4


In [0]:
np.absolute(test['price'] - test['predicted_price']).sum()

302.4

In [0]:
our_flat = pd.Series([4,2,1,23])
our_flat.index = pd.Index(features)
predicted_price = predict_price(our_flat,train)
print("We should list our flat at a nightly rate of $%d."%(predicted_price))

We should list our flat at a nightly rate of $199.
